In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [3]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

In [5]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in SF?', id='113ab5c6-4213-43dd-8f06-4f0c41654c44'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cijx4TZum36wjgv87J0XykZn', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-69b94e9b-e82b-4aed-9727-9ca04fdf550d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Cijx4TZum36wjgv87J0XykZn'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cijx4TZum36wjgv87J0XykZn', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 't

In [8]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='113ab5c6-4213-43dd-8f06-4f0c41654c44'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cijx4TZum36wjgv87J0XykZn', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-69b94e9b-e82b-4aed-9727-9ca04fdf550d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Cijx4TZum36wjgv87J0XykZn'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2442a-c1be-611f-8001-df5197e1fd50'}}, metadata={'source': 'loop', 'step': 1, 'writes':

In [9]:
abot.graph.get_state(thread).next

('action',)

In [10]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Cijx4TZum36wjgv87J0XykZn'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717705142, \'localtime\': \'2024-06-06 13:19\'}, \'current\': {\'last_updated_epoch\': 1717704900, \'last_updated\': \'2024-06-06 13:15\', \'temp_c\': 18.9, \'temp_f\': 66.0, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 12.5, \'wind_kph\': 20.2, \'wind_degree\': 290, \'wind_dir\': \'WNW\', \'pressure_mb\': 1012.0, \'pressure_in\': 29.88, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 63, \'cloud\': 0, \'feelslike_c\': 18.9, \'feelslike

In [11]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='113ab5c6-4213-43dd-8f06-4f0c41654c44'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Cijx4TZum36wjgv87J0XykZn', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-69b94e9b-e82b-4aed-9727-9ca04fdf550d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_Cijx4TZum36wjgv87J0XykZn'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \

In [12]:
abot.graph.get_state(thread).next

()

In [13]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread), "\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='b96e4fb1-5207-4403-a2a8-fd5bd2f28b01'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lftzLKmbKKl0xqSnfrgXpfDc', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cb9e4b90-b210-4b67-aca9-5930635006ac-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_lftzLKmbKKl0xqSnfrgXpfDc'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lftzLKmbKKl0xqSnfrgXpfDc', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_

In [14]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_

In [15]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef2442c-c792-6f7a-8001-447e7a303bf0'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'l

In [16]:
current_values = abot.graph.get_state(thread)

In [17]:
current_values.values["messages"][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})

In [18]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Los Angeles'},
  'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}]

In [19]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Louisiana"},
        "id": _id,
    }
]

In [20]:
abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}


{'configurable': {'thread_id': '3',
  'thread_ts': '1ef2442d-f053-6878-8002-e51adfe42ea8'}}

In [21]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef2442d-f053-6878-8002-e51adfe42ea8'}}, metadata={'source': 'update', 'step': 2, 'writ

In [22]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1717705486, \'localtime\': \'2024-06-06 15:24\'}, \'current\': {\'last_updated_epoch\': 1717704900, \'last_updated\': \'2024-06-06 15:15\', \'temp_c\': 27.4, \'temp_f\': 81.3, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 16.1, \'wind_kph\': 25.9, \'wind_degree\': 290, \'wind_dir\': \'WNW\', \'pressure_mb\': 1010.0, \'pressure_in\': 29.81, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 26, \'cloud\': 0, \'feelslike_c\': 28.5, \'feelslike_f\

In [23]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA

In [24]:
to_replay = states[-3]

In [25]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef2442c-c792-6f7a-8001-447e7a303bf0'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'l

In [26]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717705514, \'localtime\': \'2024-06-06 13:25\'}, \'current\': {\'last_updated_epoch\': 1717704900, \'last_updated\': \'2024-06-06 13:15\', \'temp_c\': 20.0, \'temp_f\': 68.0, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 10.5, \'wind_kph\': 16.9, \'wind_degree\': 250, \'wind_dir\': \'WSW\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.92, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 73, \'cloud\': 100, \'feelslike_c\': 20.0, \'feelslik

In [27]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef2442c-c792-6f7a-8001-447e7a303bf0'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'l

In [28]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
    }
]

In [29]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}


In [30]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1717705514, \'localtime\': \'2024-06-06 13:25\'}, \'current\': {\'last_updated_epoch\': 1717704900, \'last_updated\': \'2024-06-06 13:15\', \'temp_c\': 20.0, \'temp_f\': 68.0, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 10.5, \'wind_kph\': 16.9, \'wind_degree\': 250, \'wind_dir\': \'WSW\', \'pressure_mb\': 1013.0, \'pressure_in\': 29.92, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 73, \'cloud\': 100, \'feelslike_c\': 20.0

In [31]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173})]}, next=('action',), config={'configurable': {'thread_id': '3', 'thread_ts': '1ef2442c-c792-6f7a-8001-447e7a303bf0'}}, metadata={'source': 'loop', 'step': 1, '

In [32]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

In [33]:
state_update = {
    "messages": [
        ToolMessage(
            tool_call_id=_id,
            name="tavily_search_results_json",
            content="54 degree celcius",
        )
    ]
}

In [34]:
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [35]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', id='4a15be44-8c71-445e-bbdb-0c60140a09a5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t0t0V7AeijZUB2Obd2MRXavj', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b63fc2e0-6f7e-4dba-95b4-20ca86e82935-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_t0t0V7AeijZUB2Obd2MRXavj'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173}), ToolMessage(content='54 degree celcius', name='tavily_search_results_json', id='d506f042-51b6-4141-aada-62fff0724cfb', tool_call_id='call_t0t0V7AeijZUB2Obd2MRXavj'), AIMessage(content='The we

In [36]:
from dotenv import load_dotenv

_ = load_dotenv()

In [37]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

In [38]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [39]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {
        "lnode": "node_1",
        "count": 1,
    }


def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {
        "lnode": "node_2",
        "count": 1,
    }

In [40]:
def should_continue(state):
    return state["count"] < 3

In [41]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [42]:
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

In [43]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [44]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f719-68bd-8004-fa67dfd2c164'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-06T20:30:05.976982+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f713-662f-8003-a5fa0a9da356'}})

In [45]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f719-68bd-8004-fa67dfd2c164'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-06T20:30:05.976982+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f713-662f-8003-a5fa0a9da356'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f713-662f-8003-a5fa0a9da356'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-06T20:30:05.974460+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f70d-6b8c-8002-496e35b5695c'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [46]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f719-68bd-8004-fa67dfd2c164'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f713-662f-8003-a5fa0a9da356'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f70d-6b8c-8002-496e35b5695c'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f703-667e-8001-a95851301515'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f6fa-63ff-8000-1bf0cb7fd732'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f6f6-6160-bfff-14f6fcc23805'}} 0


In [47]:
states[-3]

{'configurable': {'thread_id': '1',
  'thread_ts': '1ef24438-f703-667e-8001-a95851301515'}}

In [48]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f703-667e-8001-a95851301515'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-06T20:30:05.967912+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f6fa-63ff-8000-1bf0cb7fd732'}})

In [49]:
graph.invoke(None, states[-3])

node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [50]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a5b-6940-8004-14df2015aa63'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a53-61dc-8003-205debe9bd99'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a4b-6fbe-8002-82dcf0f5d1ab'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f719-68bd-8004-fa67dfd2c164'}} 4
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f713-662f-8003-a5fa0a9da356'}} 3
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f70d-6b8c-8002-496e35b5695c'}} 2
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f703-667e-8001-a95851301515'}} 1
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f6fa-63ff-8000-1bf0cb7fd732'}} 0
{'configurable': {'thread_id': '1', 'thread_ts': '1ef24438-f6f6-6160-bfff-14f6fcc23805'}} 0


In [51]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a5b-6940-8004-14df2015aa63'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-06T20:30:41.550749+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a53-61dc-8003-205debe9bd99'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a53-61dc-8003-205debe9bd99'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-06T20:30:41.547283+00:00', parent_config={'configurable': {'thread_id': '1', 'thread_ts': '1ef2443a-4a4b-6fbe-8002-82dcf0f5d1ab'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '1', 'thread_t

In [52]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread2)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [53]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values["count"])

{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f610-62bf-8004-cc575b2e303e'}} 4
{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f609-601f-8003-b2c464144eef'}} 3
{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5ff-6ec7-8002-7118292b8313'}} 2
{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5f9-6a8c-8001-96db83fe9341'}} 1
{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5f0-6b97-8000-83f815953562'}} 0
{'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5ea-6aa1-bfff-c163777e1075'}} 0


In [54]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5f9-6a8c-8001-96db83fe9341'}}, metadata={'source': 'loop', 'step': 1, 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}}, created_at='2024-06-06T20:30:59.546153+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f5f0-6b97-8000-83f815953562'}})

In [55]:
graph.update_state(thread2, save_state.values)

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef2443b-8a87-6064-8005-7018b9292649'}}

In [56]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 5}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-8a87-6064-8005-7018b9292649'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'hi', 'count': 1}}}, created_at='2024-06-06T20:31:15.122989+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f610-62bf-8004-cc575b2e303e'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f610-62bf-8004-cc575b2e303e'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-06T20:30:59.555375+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f609-601f-8003-b2c464144eef'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '2'

In [57]:
graph.update_state(thread2, save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'thread_ts': '1ef2443b-e223-6027-8006-ce2ffcf39fd8'}}

In [58]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 6}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-e223-6027-8006-ce2ffcf39fd8'}}, metadata={'source': 'update', 'step': 6, 'writes': {'Node1': {'lnode': 'node_1', 'scratch': 'hi', 'count': 1}}}, created_at='2024-06-06T20:31:24.309480+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-8a87-6064-8005-7018b9292649'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 5}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-8a87-6064-8005-7018b9292649'}}, metadata={'source': 'update', 'step': 5, 'writes': {'Node2': {'lnode': 'node_1', 'scratch': 'hi', 'count': 1}}}, created_at='2024-06-06T20:31:15.122989+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443a-f610-62bf-8004-cc575b2e303e'}}) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable'

In [59]:
graph.invoke(None, thread2)

node2, count:6


{'lnode': 'node_2', 'scratch': 'hi', 'count': 7}

In [60]:
for state in graph.get_state_history(thread2):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 7}, next=(), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443c-3605-6084-8007-a330d74efa37'}}, metadata={'source': 'loop', 'step': 7, 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}}, created_at='2024-06-06T20:31:33.105225+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-e223-6027-8006-ce2ffcf39fd8'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 6}, next=('Node2',), config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-e223-6027-8006-ce2ffcf39fd8'}}, metadata={'source': 'update', 'step': 6, 'writes': {'Node1': {'lnode': 'node_1', 'scratch': 'hi', 'count': 1}}}, created_at='2024-06-06T20:31:24.309480+00:00', parent_config={'configurable': {'thread_id': '2', 'thread_ts': '1ef2443b-8a87-6064-8005-7018b9292649'}}) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 5}, next=(), config={'configurable': {'thread_id': '2'